In [ ]:
# install category_encoders package
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, GridSearchCV
from category_encoders import (
    CountEncoder,
    OneHotEncoder,
    OrdinalEncoder,
    TargetEncoder
)
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, randint
from sklearn.impute import KNNImputer
import seaborn as sns
from matplotlib import pyplot as plt

Mounted at /content/drive


In [ ]:
# read in the data
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training_set_features.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_set_features.csv')

In [ ]:
labels_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_set_labels.csv")

In [ ]:
# np.testing.assert_array_equal(train_df.index.values, labels_df.index.values)

In [ ]:
train_df.isnull().sum()

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

In [ ]:
bad_predictors = [
    "behavioral_large_gatherings","behavioral_outside_home","opinion_h1n1_sick_from_vacc","age_group","education",'race','sex',"income_poverty",'hhs_geo_region',"census_msa","household_adults","household_children","respondent_id"
]

In [ ]:
train_df = train_df.drop(columns = bad_predictors)
test_df = test_df.drop(columns = bad_predictors)

In [ ]:
# # #eliminate null values

# # For float types
# float_cols = train_df.select_dtypes(include=[np.float]).columns
# imputer = KNNImputer(n_neighbors=5)
# train_df_float_imputed = pd.DataFrame(imputer.fit_transform(train_df[float_cols]), columns=float_cols)
# train_df[float_cols] = train_df_float_imputed

# # For string types
# train_df = train_df.fillna('No Category')

<ipython-input-101-1f99f72db51f>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_cols = train_df.select_dtypes(include=[np.float]).columns


In [ ]:
# # For float types
# float_cols = test_df.select_dtypes(include=[np.float]).columns
# imputer = KNNImputer(n_neighbors=5)
# test_df_float_imputed = pd.DataFrame(imputer.fit_transform(test_df[float_cols]), columns=float_cols)
# test_df[float_cols] = test_df_float_imputed

# # For string types
# test_df = test_df.fillna('No Category')

<ipython-input-102-d3b30328ab84>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_cols = test_df.select_dtypes(include=[np.float]).columns


###Data understanding
We are interested in predicting high probability (likely cases)

target = 'labels_df'

metric = 'roc_auc'

In [ ]:
# check the labels
labels_df.h1n1_vaccine.value_counts(normalize=True)

0    0.787546
1    0.212454
Name: h1n1_vaccine, dtype: float64

In [ ]:
labels_df.seasonal_vaccine.value_counts(normalize=True)

0    0.534392
1    0.465608
Name: seasonal_vaccine, dtype: float64

In [ ]:
labels_df=labels_df.iloc[:,[1,2]]

In [ ]:
#object_cols = train_df.columns[train_df.dtypes == "object"].values

###Modeling

In [ ]:
#for float types
train_df = train_df.fillna(train_df.mean())

#for string types
train_df = train_df.fillna('No Category')

<ipython-input-8-ff17c02b30d1>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_df = train_df.fillna(train_df.mean())


In [ ]:
test_df = test_df.fillna(test_df.mean())
test_df = test_df.fillna('No Category')

<ipython-input-9-7051169c5567>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test_df = test_df.fillna(test_df.mean())


In [ ]:
X, y = train_df,labels_df['h1n1_vaccine']
X_test = test_df

In [ ]:
# X_encoded = count_enc.fit_transform(X)
# X_scaled = scaler.fit_transform(X_encoded)
# X_test_encoded = count_enc.transform(X_test)
# X_test_scaled = scaler.transform(X_test_encoded)

###pipeline

In [ ]:
rf_params = {
    'ml__max_depth': randint(10, 13)
}

ml_pipeline = Pipeline(steps=[
    ('encoder', TargetEncoder()),
    #('scaler', StandardScaler()),
    ('ml', RandomForestClassifier())
])

In [ ]:
# Create the RandomizedSearchCV instance with the VotingClassifier
randomized_rf = RandomizedSearchCV(
    ml_pipeline,
    rf_params,
    cv=10,
    n_iter=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1,
    random_state=42,
    error_score='raise'
)

In [ ]:
# Fit the RandomizedSearchCV on X and y
randomized_rf.fit(X, y)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


RandomizedSearchCV(cv=10, error_score='raise',
                   estimator=Pipeline(steps=[('encoder', TargetEncoder()),
                                             ('ml', RandomForestClassifier())]),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'ml__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f466d073730>},
                   random_state=42, scoring='roc_auc', verbose=1)

In [ ]:
lgbm_params = {
    'ml__max_depth': randint(10, 13)
}

ml_pipeline = Pipeline(steps=[
    ('encoder', TargetEncoder()),
    #('scaler', StandardScaler()),
    ('ml', LGBMClassifier())
])

In [ ]:
# Create the RandomizedSearchCV instance with the VotingClassifier
randomized_lgbm = RandomizedSearchCV(
    ml_pipeline,
    lgbm_params,
    cv=10,
    n_iter=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1,
    random_state=42,
    error_score='raise'
)

In [ ]:
# Fit the RandomizedSearchCV on X and y
randomized_lgbm.fit(X, y)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


RandomizedSearchCV(cv=10, error_score='raise',
                   estimator=Pipeline(steps=[('encoder', TargetEncoder()),
                                             ('ml', LGBMClassifier())]),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'ml__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f462f6c6ad0>},
                   random_state=42, scoring='roc_auc', verbose=1)

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('lgbm', randomized_lgbm), ('rf', randomized_rf)],
    voting='soft'
)

In [ ]:
# Perform cross-validation on the best estimator
cross_val_scores1 = cross_val_score(
    voting_clf,
    X,
    y,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.5min finished


In [ ]:
# # Perform cross-validation on the best estimator
# cross_val_scores = cross_val_score(
#     randomized_rf,
#     X,
#     y,
#     scoring='roc_auc',
#     cv=5,
#     n_jobs=-1,
#     verbose=1
# )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.0min finished


In [ ]:
voting_clf = voting_clf.fit(X,y)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [ ]:
np.mean(cross_val_scores1)

0.8632680110007666

In [ ]:
# Make predictions using predict_proba
y_pred = voting_clf.predict_proba(X_test)

### Target = seasonal_vaccine

In [ ]:
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1_submission.csv')

In [ ]:
submission_df["h1n1_vaccine"] = y_pred[:, 1]

In [ ]:
submission_df.to_csv('/content/drive/MyDrive/Colab Notebooks/1_submission.csv', index=True)

In [ ]:
y_pred[:, 1]

array([0.15992979, 0.01531871, 0.1455424 , ..., 0.21425382, 0.04856303,
       0.65687774])